# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [46]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [47]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lazaro cardenas,17.9583,-102.2000,70.03,75,23,3.83,MX,1738322074
1,1,lihue,21.9789,-159.3672,72.99,78,0,23.02,US,1738322075
2,2,grytviken,-54.2811,-36.5092,48.47,75,94,16.80,GS,1738322076
3,3,dudinka,69.4058,86.1778,-8.05,97,100,14.00,RU,1738322077
4,4,puerto ayora,-0.7393,-90.3518,75.54,98,66,2.84,EC,1738322078
...,...,...,...,...,...,...,...,...,...,...
576,576,kamaishi,39.2667,141.8833,29.59,66,25,8.88,JP,1738322817
577,577,manismata,-2.4667,111.0333,77.52,88,59,4.83,ID,1738322818
578,578,poli,-3.3333,36.8000,74.07,51,92,2.80,TZ,1738322819
579,579,ingeniero jacobacci,-41.3427,-69.5448,57.78,61,0,11.77,AR,1738322820


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [48]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values


city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 79) & (city_data_df["Max Temp"] > 20)]



# #Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]


#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,puerto el triunfo,13.2833,-88.5500,75.67,69,0,4.21,SV,1738322092
111,111,san pedro pochutla,15.7333,-96.4667,71.71,73,0,3.47,MX,1738322203
118,118,alpine,32.8350,-116.7664,41.86,92,0,0.00,US,1738322211
197,197,brisas de zicatela,15.8369,-97.0419,75.42,71,0,4.45,MX,1738322386
235,235,linares,-35.8500,-71.6000,50.74,72,0,3.60,CL,1738322430
270,270,andalgala,-27.5819,-66.3165,76.33,49,0,1.92,AR,1738322470
317,317,dibai,28.2167,78.2500,76.71,35,0,3.60,IN,1738322524
357,357,pahasu,28.1833,78.0500,76.35,36,0,3.65,IN,1738322570
403,403,mont-de-marsan,44.0000,-0.6667,52.54,87,0,2.30,FR,1738322623
425,425,indaparapeo,19.7833,-100.9667,39.88,81,0,0.00,MX,1738322647


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,puerto el triunfo,SV,13.2833,-88.5500,69,
111,san pedro pochutla,MX,15.7333,-96.4667,73,
118,alpine,US,32.8350,-116.7664,92,
197,brisas de zicatela,MX,15.8369,-97.0419,71,
235,linares,CL,-35.8500,-71.6000,72,
270,andalgala,AR,-27.5819,-66.3165,49,
317,dibai,IN,28.2167,78.2500,35,
357,pahasu,IN,28.1833,78.0500,36,
403,mont-de-marsan,FR,44.0000,-0.6667,87,
425,indaparapeo,MX,19.7833,-100.9667,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address =  name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto el triunfo - nearest hotel: Motel Jardín
san pedro pochutla - nearest hotel: Olimpia
alpine - nearest hotel: Ayres Lodge Alpine
brisas de zicatela - nearest hotel: Casa de Olas
linares - nearest hotel: Hotel Curapalihue
andalgala - nearest hotel: Residencial Ayucaba
dibai - nearest hotel: No hotel found
pahasu - nearest hotel: No hotel found
mont-de-marsan - nearest hotel: No hotel found
indaparapeo - nearest hotel: Hotel GM Soria
los frentones - nearest hotel: No hotel found
sanya - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,puerto el triunfo,SV,13.2833,-88.5500,69,Motel Jardín
111,san pedro pochutla,MX,15.7333,-96.4667,73,Olimpia
118,alpine,US,32.8350,-116.7664,92,Ayres Lodge Alpine
197,brisas de zicatela,MX,15.8369,-97.0419,71,Casa de Olas
235,linares,CL,-35.8500,-71.6000,72,Hotel Curapalihue
270,andalgala,AR,-27.5819,-66.3165,49,Residencial Ayucaba
317,dibai,IN,28.2167,78.2500,35,No hotel found
357,pahasu,IN,28.1833,78.0500,36,No hotel found
403,mont-de-marsan,FR,44.0000,-0.6667,87,No hotel found
425,indaparapeo,MX,19.7833,-100.9667,81,Hotel GM Soria


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)